# Estimation II
## Practical Exercises

### Exercise 7
  In this exercise, we will compare $\ell_1$ and $\ell_2$ regularization for image denoising using wavelet transform.

  One way to view an image is as a function $f(i,j)$ on the pixel position $(i,j)$. Using wavelet transform, we can represent the function (image) as $f(i,j) = \langle \mathbf{w}, \phi(i,j) \rangle$ where $\mathbf{w}$ are the linear function weights (wavelet coefficients) and $\phi(i,j)$ is the feature vector formed by the inverse wavelet transform bases. If we add noise to each pixel and try to learn a predictor (weight vector $\mathbf{w}$), we have a regression problem.

For this problem, it is simpler to think of an image as a vector $\mathbf{v}$ in $\mathbb{R}^{n}$, where $n$ is the number of pixels. The $i$-th coefficient of the wavelet transform is then the projection of $\mathbf{v}$ on the $i$-th basis. Because wavelet is an orthonormal transform, it preserves distance (see background material on linear algebra), i.e. if $\mathbf{\bar{w}}$ is an estimate of $\mathbf{w}$ that gives an estimate $\mathbf{\bar{v}}$ of $\mathbf{v}$, then $\|\mathbf{v}-\mathbf{\bar{v}}\|= \|\mathbf{w}-\mathbf{\bar{w}}\|$ or $\sum_{i} (v_i-\bar{v}_i)^2 = \sum_{i} (w_i-\bar{w}_i)^2$. This makes the implementing $\ell_1$ and $\ell_2$ regularization easy.

For $\ell_2$ regularization, $\sum_{i} (v_i-\bar{v}_i)^2 + \lambda \sum_i \bar{w}_i^2 = \sum_{i} (w_i-\bar{w}_i)^2 + \lambda \sum_i \bar{w}_i^2$. Each coefficient $\bar{w}_i$ can be optimized independently giving the solution $\bar{w}_i=\frac{1}{1+\lambda}w_i$, or each coefficient is multiplied by a constant factor smaller than 1.

For $\ell_1$ regularization, $\sum_{i} (v_i-\bar{v}_i)^2 + \lambda \sum_i |\bar{w}_i| = \sum_{i} (w_i-\bar{w}_i)^2 + \lambda \sum_i |\bar{w}_i|$. Each coefficient can again be optimized independently, giving the soft thresholding function as the solution: if $|w_i|$ is smaller than $\lambda$, set $\bar{w}_i$ to 0, otherwise set $\bar{w}_i$ to $sign(w_i)(|w_i|-\lambda)$, i.e. reduce the size of the coefficient by $\lambda$ while keeping the same sign.

Run the experiment comparing $\ell_1$ and $\ell_2$ regularization. Performance in signal processing is commonly measured using signal to noise ratio: $SNR(\mathbf{p},\bar{\mathbf{p}}) = 10\log_{10} \frac{\|\mathbf{p}\|^2}{\|\mathbf{p}-\bar{\mathbf{p}}\|^2}$ (the larger the better). Try other images such as boat.png, flowers.png and mandrill.png. Try varying the soft threshold and ridge parameters.

***Archipelago:*** Which regularization method performs better for wavelet denoising? Why?

The code is modified from ** Numerical Tours in Python ** http://www.numerical-tours.com/python/ .

In [ ]:
from __future__ import division
from nt_toolbox.general import *
from nt_toolbox.signal import *
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

%pylab inline

# Load image
n = 512
name = 'nt_toolbox/data/boat.png'
f0 = load_image(name, n)
imageplot(f0, 'Original image')
show()

# Add noise
sigma = .08
f = f0 + sigma*random.standard_normal(f0.shape)
imageplot(clamp(f), 'Noisy, SNR=' + str(snr(f0,f)) )
show()

# Do wavelet tranform
h = [0, .482962913145, .836516303738, .224143868042, -.129409522551]
h = h/norm(h)
Jmin = 2
a = perform_wavortho_transf(f,Jmin,+1,h)
plot_wavelet(a,Jmin);
print "Wavelet Coefficients"
show()

# Soft thresholding, equivalent to l_1 regularization
def thresh_soft(u,t):return maximum(1-t/abs(u), 0)*u
alpha = linspace(-3,3,1000)
plot(alpha, thresh_soft(alpha,1))
axis('equal');
print "Soft Thresholding Function"
show()
T = 1.4*sigma
aT = thresh_soft(a,T)
fSoft = perform_wavortho_transf(aT,Jmin,-1,h)
imageplot(clamp(fSoft), 'Soft, SNR=' + str(snr(f0,fSoft)) )
show()

# Ridge regression, equivalent to l_2 regularization
aR = 0.98*a
fRidge = perform_wavortho_transf(aR,Jmin,-1,h)
imageplot(clamp(fRidge), 'Ridge, SNR=' + str(snr(f0,fRidge)) )
show()

# Mean squared error and fraction of features removed
mseR = mean_squared_error(f0,fRidge)
mseS = mean_squared_error(f0,fSoft)
print "MSE soft thresholding: " + "{0:.4f}".format(mseS)
print "MSE ridge regression: " + "{0:.4f}".format(mseR)
softFrac = sum(aT==0)/(sum(aT!=0)+sum(aT==0))
ridgeFrac = sum(aR==0)/(sum(aR!=0)+sum(aR==0))
print "Fraction of zero coeffs in soft thresholding: " + "{0:.2f}".format(softFrac)
print "Fraction of zero coeffs in ridge regression: " + "{0:.2f}".format(ridgeFrac)

### Exercise 8
Demo on bagging from http://scikit-learn.org/stable/auto_examples/ensemble/plot_bias_variance.html

In [ ]:
print(__doc__)

# Author: Gilles Louppe <g.louppe@gmail.com>
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

# Settings
n_repeat = 50       # Number of iterations for computing expectations
n_train = 50        # Size of the training set
n_test = 1000       # Size of the test set
noise = 0.1         # Standard deviation of the noise
np.random.seed(0)

# Change this for exploring the bias-variance decomposition of other
# estimators. This should work well for estimators with high variance (e.g.,
# decision trees or KNN), but poorly for estimators with low variance (e.g.,
# linear models).
estimators = [("Tree", DecisionTreeRegressor()),
              ("Bagging(Tree)", BaggingRegressor(DecisionTreeRegressor()))]

n_estimators = len(estimators)

# Generate data
def f(x):
    x = x.ravel()

    return np.exp(-x ** 2) + 1.5 * np.exp(-(x - 2) ** 2)

def generate(n_samples, noise, n_repeat=1):
    X = np.random.rand(n_samples) * 10 - 5
    X = np.sort(X)

    if n_repeat == 1:
        y = f(X) + np.random.normal(0.0, noise, n_samples)
    else:
        y = np.zeros((n_samples, n_repeat))

        for i in range(n_repeat):
            y[:, i] = f(X) + np.random.normal(0.0, noise, n_samples)

    X = X.reshape((n_samples, 1))

    return X, y

X_train = []
y_train = []

for i in range(n_repeat):
    X, y = generate(n_samples=n_train, noise=noise)
    X_train.append(X)
    y_train.append(y)

X_test, y_test = generate(n_samples=n_test, noise=noise, n_repeat=n_repeat)

# Loop over estimators to compare
for n, (name, estimator) in enumerate(estimators):
    # Compute predictions
    y_predict = np.zeros((n_test, n_repeat))

    for i in range(n_repeat):
        estimator.fit(X_train[i], y_train[i])
        y_predict[:, i] = estimator.predict(X_test)

    # Bias^2 + Variance + Noise decomposition of the mean squared error
    y_error = np.zeros(n_test)

    for i in range(n_repeat):
        for j in range(n_repeat):
            y_error += (y_test[:, j] - y_predict[:, i]) ** 2

    y_error /= (n_repeat * n_repeat)

    y_noise = np.var(y_test, axis=1)
    y_bias = (f(X_test) - np.mean(y_predict, axis=1)) ** 2
    y_var = np.var(y_predict, axis=1)

    print("{0}: {1:.4f} (error) = {2:.4f} (bias^2) "
          " + {3:.4f} (var) + {4:.4f} (noise)".format(name,
                                                      np.mean(y_error),
                                                      np.mean(y_bias),
                                                      np.mean(y_var),
                                                      np.mean(y_noise)))
    plt.figure(1, figsize=(12, 9))
    # Plot figures
    plt.subplot(2, n_estimators, n + 1)
    plt.plot(X_test, f(X_test), "b", label="$f(x)$")
    plt.plot(X_train[0], y_train[0], ".b", label="LS ~ $y = f(x)+noise$")

    for i in range(n_repeat):
        if i == 0:
            plt.plot(X_test, y_predict[:, i], "r", label="$\^y(x)$")
        else:
            plt.plot(X_test, y_predict[:, i], "r", alpha=0.05)

    plt.plot(X_test, np.mean(y_predict, axis=1), "c",
             label="$\mathbb{E}_{LS} \^y(x)$")

    plt.xlim([-5, 5])
    plt.title(name)

    if n == 0:
        plt.legend(loc="upper left", prop={"size": 11})

    plt.subplot(2, n_estimators, n_estimators + n + 1)
    plt.plot(X_test, y_error, "r", label="$error(x)$")
    plt.plot(X_test, y_bias, "b", label="$bias^2(x)$"),
    plt.plot(X_test, y_var, "g", label="$variance(x)$"),
    plt.plot(X_test, y_noise, "c", label="$noise(x)$")

    plt.xlim([-5, 5])
    plt.ylim([0, 0.1])

    if n == 0:
        plt.legend(loc="upper left", prop={"size": 11})

plt.show()